In [1]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [2]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static
    
def generate_grid(filename):
    
    face = open(filename+'.face','r').read()
    vert = open(filename+'.vert','r').read()

    faces = numpy.vstack(numpy.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
    verts = numpy.vstack(numpy.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

    grid = bempp.api.grid_from_element_data(verts.transpose(), faces.transpose())
    
    N = grid.leaf_view.entity_count(0)
    elements = list(grid.leaf_view.entity_iterator(0))
    area = numpy.zeros(N)

    # remove zero areas
    for i in range(N):
        area[i] = elements[i].geometry.volume

    area_nonzero = numpy.where(area>1e-12)[0]
    
    faces_nonzero = faces[area_nonzero,:]
    
    grid = bempp.api.grid_from_element_data(verts.transpose(), faces_nonzero.transpose())
    
    return grid

def energy_mol(mesh_diel, mesh_stern, phi_static, eps_s):

    grid_diel = generate_grid(mesh_diel)
    grid_stern = generate_grid(mesh_stern)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([eps_s*phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    

    M11   = laplace.single_layer(space_diel, space_diel, space_diel) 
    M12   = laplace.single_layer(space_stern, space_diel, space_diel) 
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern) 

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, tol=1e-5, maxiter=500, restart = 1000)
    
    if info>0:
        print 'Not converged, %i iterations'%info
    elif info<0:
        print 'Solver diverges'
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]

    
    qe = 1.60217662e-19
    Rw = 1.4
    Na = 6.0221409e23
    eps_0 = 8.854187817e-12
    
    #water_charge = 0.83*qe/(numpy.pi*Rw**2) * Na*qe*1e10/(1000*eps_0)
    water_dipole = 6.17e-30/(1e-10)/((2*Rw)**2) * Na*qe*1e10/(1000*eps_0)
    print water_dipole
    print sigma_s[:5]*1.4
    print max(abs(sigma_s*1.4))
    '''
    test_charge = numpy.where(abs(sigma_d)*1.4>water_dipole)[0]
    #print abs(sigma_d)
    if len(test_charge)>0:
        print 'Unphysical charge in dielectric surface'
        print len(test_charge)
    '''    
    
    test_charge = numpy.where(abs(sigma_s)*1.4>water_dipole)[0]
    if len(test_charge)>0:
        print 'Unphysical charge in stern surface'
        print len(test_charge)
    
    area_d = numpy.zeros(N_diel)
    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)

    return energy, float(len(test_charge))/N_stern

In [3]:
phi_static = 10.7 #kcal/mol/e
phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 6.65

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(problem_folder+namefile))
energy = numpy.zeros(num_lines)
test_charge_arr = numpy.zeros(num_lines)
it = 0
for line in file(problem_folder+namefile):
    mol = line[:-1]

    mesh_stern = problem_folder+mol+'/surf_d02_stern_fix'
    mesh_diel = problem_folder+mol+'/surf_d02_fix'

    try: 
        e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
    except:
        print 'Something went wrong'
        e = 0
        test_charge = 0.
            
    ekJ = e*conv_factor # kJ/mol
    ekcal = ekJ/4.184   # kcal/mol
    
    energy[it] = ekcal
    test_charge_arr[it] = test_charge
    
    print 'Run %i for '%it + mol + '. Energy = %1.3f kcal/mol'%energy[it]
    
    it += 1
    
numpy.savetxt('energy_bem.txt', energy)
numpy.savetxt('test_charge.txt', test_charge_arr)

857.593881087
[  120.71706995 -2267.23016236  -675.12837553  -879.86332508  -853.11760826]
19267.0216022
Unphysical charge in stern surface
121
Run 0 for 1112_tetrachloroethane. Energy = 1.103 kcal/mol
857.593881087
[ -768.19823107 -2610.14821133 -1245.94384144  -803.30276344  -957.76462257]
22041.2295461
Unphysical charge in stern surface
103
Run 1 for 111_trichloroethane. Energy = 1.019 kcal/mol
857.593881087
[  35.04193664 -469.66981167 -161.47597988  217.26372259  130.83268672]
30511.493028
Unphysical charge in stern surface
159
Run 2 for 111_trifluoro_222_trimethoxyethane. Energy = 1.317 kcal/mol
857.593881087
[ -500.5814579  -2609.46984789 -1251.60075893  -892.3670052  -1027.78106387]
18151.4255367
Unphysical charge in stern surface
119
Run 3 for 111_trifluoropropan_2_ol. Energy = 0.998 kcal/mol
857.593881087
[ 168.08338109 -141.06949186 -151.98116014  191.09400512  -53.82658247]
37263.6729607
Unphysical charge in stern surface
144
Run 4 for 111_trimethoxyethane. Energy = 1.236 k

857.593881087
[ -180.37266114 -2584.57461913 -2625.90250951  -263.6596818   -526.1796966 ]
16462.3841061
Unphysical charge in stern surface
188
Run 42 for 1_bromohexane. Energy = 1.416 kcal/mol
857.593881087
[-459.78427729 -694.58843508 -882.91909474 -166.88811731 -363.23325923]
20149.4909774
Unphysical charge in stern surface
258
Run 43 for 1_bromooctane. Energy = 1.696 kcal/mol
857.593881087
[-429.97598546 -695.50505604 -517.02937592 -548.63596384 -558.74131215]
17955.6965655
Unphysical charge in stern surface
163
Run 44 for 1_bromopentane. Energy = 1.272 kcal/mol
857.593881087
[-403.86810806 -405.64259098 -250.41910236 -378.15091437 -197.86213002]
17638.7443867
Unphysical charge in stern surface
102
Run 45 for 1_bromopropane. Energy = 0.988 kcal/mol
857.593881087
[ -618.69030887 -1983.84621447  -813.53245134  -712.1077734  -1054.25007814]
17672.4364414
Unphysical charge in stern surface
67
Run 46 for 1_chloro_222_trifluoroethane. Energy = 0.929 kcal/mol
857.593881087
[ -316.02570602

857.593881087
[  -3.85058684 -137.48900236   18.37715634  292.48373052 -519.36495109]
40009.7902883
Unphysical charge in stern surface
161
Run 84 for 25_dimethylphenol. Energy = 1.304 kcal/mol
857.593881087
[-376.77159659 -693.08893421 -636.97012037 -423.55657383 -636.14316761]
17084.5067345
Unphysical charge in stern surface
162
Run 85 for 25_dimethylpyridine. Energy = 1.241 kcal/mol
857.593881087
[-603.09203176 -573.43639567 -698.46134233 -298.97983425 -338.04538974]
20351.1944702
Unphysical charge in stern surface
171
Run 86 for 25_dimethyltetrahydrofuran. Energy = 1.206 kcal/mol
857.593881087
[ -45.2999736    -2.32616593  -11.64668037 -176.95785383 -119.92153677]
27202.2037063
Unphysical charge in stern surface
205
Run 87 for 26_dimethylaniline. Energy = 1.305 kcal/mol
857.593881087
[ -436.65756634  -608.38648584  -272.43853584  -217.20329451 -1322.88499112]
20255.2655325
Unphysical charge in stern surface
303
Run 88 for 26_dimethylnaphthalene. Energy = 1.579 kcal/mol
857.593881087

857.593881087
[  247.15349165 -2093.55826157  -910.70695647  -746.06908162 -1091.97212312]
17391.8028815
Unphysical charge in stern surface
115
Run 126 for 2_methylpropan_1_ol. Energy = 1.009 kcal/mol
857.593881087
[ -424.06654146 -1913.10758114  -941.01619439  -977.76647104  -973.64536556]
17001.7913579
Unphysical charge in stern surface
119
Run 127 for 2_methylpropan_2_ol. Energy = 1.009 kcal/mol
857.593881087
[ -821.00934938 -1185.49124064  -818.23717653  -948.4924946  -2215.98782015]
18460.7264735
Unphysical charge in stern surface
114
Run 128 for 2_methylpropane. Energy = 0.977 kcal/mol
857.593881087
[ -992.9826888  -1996.16490762  -806.76825664   511.41386835  -856.80583245]
32932.6308054
Unphysical charge in stern surface
164
Run 129 for 2_methylpropene. Energy = 0.943 kcal/mol
857.593881087
[-375.86174485 -839.88369067 -582.85424615 -540.03817337 -726.39822272]
54324.3462157
Unphysical charge in stern surface
121
Run 130 for 2_methylpyrazine. Energy = 1.056 kcal/mol
857.5938810

857.593881087
[ -860.89617215 -1130.52942408  -751.44271941 -1020.83523364 -1995.24548115]
18489.8888714
Unphysical charge in stern surface
211
Run 168 for 3_methylheptane. Energy = 1.499 kcal/mol
857.593881087
[-295.81884593 -416.64124018 -428.17374504 -532.34663163 -546.68375527]
680867.469142
Unphysical charge in stern surface
191
Run 169 for 3_methylhexane. Energy = 1.337 kcal/mol
857.593881087
[-185.701283   -730.64731987 -241.28692632 -823.93754008 -594.49506092]
17300.0432945
Unphysical charge in stern surface
137
Run 170 for 3_methylpentane. Energy = 1.216 kcal/mol
857.593881087
[ 589.3123427  -690.02838828 -224.78284257  189.32503337 -130.30764275]
17689.195133
Unphysical charge in stern surface
126
Run 171 for 3_methylpyridine. Energy = 1.090 kcal/mol
857.593881087
[ -733.21052059 -1375.76837444  -899.38614517 -1010.52266421  -909.61184567]
101027.369115
Unphysical charge in stern surface
125
Run 172 for 3_nitroaniline. Energy = 1.217 kcal/mol
857.593881087
[-1018.00187469 -1

...

857.593881087
[ -359.83731486   453.30541901  -626.82291394 -1317.09301015  -832.0195151 ]
30994.3447851
Unphysical charge in stern surface
201
Run 296 for diisopropylamine. Energy = 1.299 kcal/mol
857.593881087
[ 229.08455067 -884.6699722  -118.82896459  659.4622077    -6.38372712]
24737.2284843
Unphysical charge in stern surface
101
Run 297 for dimethoxymethane. Energy = 0.929 kcal/mol
857.593881087
[-1112.26007297 -1085.63635622  -939.936318     -79.68628208 -1114.89288529]
49129.2288095
Unphysical charge in stern surface
95
Run 298 for dimethyl_disulfide. Energy = 0.975 kcal/mol
857.593881087
[ -630.96023889  -615.80637005  -526.6860925   -724.35980575 -1103.08964396]
20433.9685111
Unphysical charge in stern surface
48
Run 299 for dimethyl_ether. Energy = 0.745 kcal/mol
857.593881087
[ 256.39557476  -50.35551933   97.12620429   93.02127334  102.9565182 ]
21055.6368685
Unphysical charge in stern surface
126
Run 300 for dimethyl_sulfate. Energy = 1.088 kcal/mol
857.593881087
[  32.10

857.593881087
[ -864.40713775 -1153.21958678  -917.5635773   -792.37310688  -869.54553055]
15843.8295241
Unphysical charge in stern surface
188
Run 339 for hexanoic_acid. Energy = 1.343 kcal/mol
857.593881087
[-341.96486077 -337.30675862   15.82844411   83.33263669 -333.83160922]
433.378223581
Run 340 for hydrazine. Energy = 0.523 kcal/mol
Something went wrong
Run 341 for hydrogen_sulfide. Energy = 0.000 kcal/mol
857.593881087
[-363.4608124  -706.35500655 -621.99379081 -773.01533196 -818.11354531]
20550.609137
Unphysical charge in stern surface
37
Run 342 for imidazole. Energy = 0.804 kcal/mol
857.593881087
[ -669.70135797 -1884.04546448  -992.06763651 -1095.88175289  -809.56212098]
73096.0410902
Unphysical charge in stern surface
219
Run 343 for indane. Energy = 1.285 kcal/mol
857.593881087
[-1021.97007495  -823.89801216 -1089.95388903  -580.67302755  -990.67893557]
27640.0075151
Unphysical charge in stern surface
174
Run 344 for iodobenzene. Energy = 1.175 kcal/mol
857.593881087
[-40

857.593881087
[ -910.99743511 -1099.88818173  -755.96533293  -674.74972014 -1049.17730295]
25067.4787708
Unphysical charge in stern surface
203
Run 382 for methyl_p_methoxybenzoate. Energy = 1.515 kcal/mol
857.593881087
[-1003.20164208 -1201.8410577   -391.10812938 -1303.99493141  -579.58614172]
70317.4203844
Unphysical charge in stern surface
200
Run 383 for methyl_p_nitrobenzoate. Energy = 1.497 kcal/mol
857.593881087
[-10876.3039136    8599.86615496   -750.47749636  -1408.48774981
   -773.06225868]
44700.7805818
Unphysical charge in stern surface
197
Run 384 for methyl_pentanoate. Energy = 1.359 kcal/mol
857.593881087
[ 862.77796522 -635.85710824  423.37694091 -311.16807967  230.57948488]
23722.6266
Unphysical charge in stern surface
139
Run 385 for methyl_propanoate. Energy = 1.074 kcal/mol
857.593881087
[ -654.40206788 -1055.65540159  -839.67986327  -886.05081602 -1793.77463034]
25566.613108
Unphysical charge in stern surface
142
Run 386 for methyl_propyl_ether. Energy = 1.062 kca

857.593881087
[ -609.82512921 -2147.70520971  -595.76007044  -722.18513132  -627.822979  ]
25812.4869021
Unphysical charge in stern surface
203
Run 424 for n_propylcyclopentane. Energy = 1.392 kcal/mol
857.593881087
[ 1411.82899449  -731.09700251    47.69136493  -202.42460785   130.6441348 ]
35559.5536166
Unphysical charge in stern surface
237
Run 425 for naphthalene. Energy = 1.302 kcal/mol
857.593881087
[-1039.40661583  -793.0451937   -938.07464133  -689.88804513 -1124.60615527]
106490.147879
Unphysical charge in stern surface
135
Run 426 for nitrobenzene. Energy = 1.153 kcal/mol
857.593881087
[-328.59839143 -632.76538323 -386.58126373 -146.1801968  -338.31753096]
34713.5256475
Unphysical charge in stern surface
86
Run 427 for nitroethane. Energy = 0.876 kcal/mol
857.593881087
[ 295.68874244 -162.342983   -196.08820453   69.22123591 -147.85013414]
117101.676828
Unphysical charge in stern surface
39
Run 428 for nitromethane. Energy = 0.725 kcal/mol
857.593881087
[-593.22207302 -492.80

857.593881087
[ -651.6539857  -1583.40848146  -751.59045807  -870.30209542  -695.83721037]
17803.5539719
Unphysical charge in stern surface
124
Run 467 for propan_2_ol. Energy = 0.893 kcal/mol
857.593881087
[-501.2515996  -635.77645979 -491.05376931 -551.16396505 -516.49664556]
32184.8305867
Unphysical charge in stern surface
82
Run 468 for propane. Energy = 0.858 kcal/mol
857.593881087
[ -343.22207843   537.90601952  -927.25658686 -1269.20350277  -796.69013968]
21019.5368864
Unphysical charge in stern surface
74
Run 469 for propanenitrile. Energy = 0.874 kcal/mol
857.593881087
[-128.64213291 -274.54109615 -455.22711809  192.18137185  141.27216759]
22285.5159751
Unphysical charge in stern surface
100
Run 470 for propanoic_acid. Energy = 0.906 kcal/mol
857.593881087
[ 245.2604196  -115.31713437 -317.78497826   90.76491748 -559.52238818]
16856.8736611
Unphysical charge in stern surface
106
Run 471 for propanone. Energy = 0.867 kcal/mol
857.593881087
[  764.68034041  1091.05426338   175.5

In [12]:
mesh_stern = 'mobley_test/nitroethane/surf_d02_stern'
mesh_diel = 'mobley_test/nitroethane/surf_d02'

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 5.5#80.

phi_static = 46. #kJ/mol/e
conv_factor = (1000*eps_0)/(qe**2*Na*1e10)


e, test_charge = energy_mol(mesh_diel, mesh_stern, phi_static, eps_s)
ekj = e * conv_factor#/ ((qe**2*Na*1e10)/(1000*eps_0))
ekcal = ekj/4.184
#print e, e / ((qe**2*Na*1e10)/(1000*eps_0)), e/(1000*4.184) #kJ/mol
print ekcal

857.593881087
[-278.6805626  -496.77562928 -318.64588094 -114.92787056 -278.07517599]
28648.1780082
Unphysical charge in stern surface
45
8.82394356584


In [33]:
grid_diel = generate_grid(mesh_diel)
grid_stern = generate_grid(mesh_stern)

space_diel = bempp.api.function_space(grid_diel, "DP", 0)
space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
N_diel = grid_diel.leaf_view.entity_count(0)
N_stern = grid_stern.leaf_view.entity_count(0)
    
elements_d = list(grid_diel.leaf_view.entity_iterator(0))
elements_s = list(grid_stern.leaf_view.entity_iterator(0))

print elements_d[0].geometry.corners
print elements_d[0].geometry.local2global(numpy.array([[1.,0.,0.]]))

[[-0.802 -0.885 -0.46 ]
 [ 1.119  0.807  1.406]
 [ 0.339  0.758  0.614]]


AttributeError: 'Geometry' object has no attribute 'global2local'

In [20]:
F = 1000*eps_0/(qe**2*Na*1e10)
1/(4*numpy.pi*F*4.184)
9081457.48434/(332.063*4*numpy.pi)
(qe**2*Na*1e10)/(eps_0)
2353.39653991/40

58.834913497749994

In [19]:
# Check (1/2+V)phi_1 is zero to avoid induced charge

def compute_induced_charge_term(mesh):
    
    grid = generate_grid(mesh)
    space = bempp.api.function_space(grid, "DP", 0)
    
    N = grid.leaf_view.entity_count(0)
    phi_static = 10.
    phi_static *= 4.184
    
    dirichlet_fun = bempp.api.GridFunction(space, coefficients=numpy.ones(N)*phi_static)

    identity = sparse.identity(space, space, space)
    dlp      = laplace.double_layer(space, space, space)

    induced_charge_term = (.5 * identity + dlp) * dirichlet_fun
    
    return numpy.max(numpy.abs(induced_charge_term.coefficients))
    
    
#phi_static = 10. #kcal/mol/e
#phi_static *= 4.184 # kJ/mol/e

qe = 1.60217662e-19
Na = 6.0221409e23
eps_0 = 8.854187817e-12
eps_s = 6.2

conv_factor = (1000*eps_0)/(qe**2*Na*1e10)

problem_folder = 'mobley_test/'
namefile = 'actually_all_names.txt'
num_lines = sum(1 for line in open(namefile))
induced_charge_term_array = numpy.zeros(num_lines)
it = 0
for line in file(namefile):
    
    mol = line[:-1]
    mesh = problem_folder+mol+'/surf_d02_fix'
    print 'Run %i for '%it + mol 
    
    try: 
        induced_charge_term = compute_induced_charge_term(mesh)
    except:
        print 'Something went wrong'
        induced_charge_term = 0

    
    induced_charge_term_array[it] = induced_charge_term    
    
    it += 1
    
print numpy.max(induced_charge_term_array)

Run 0 for 1112_tetrachloroethane
Run 1 for 111_trichloroethane
Run 2 for 111_trifluoro_222_trimethoxyethane
Run 3 for 111_trifluoropropan_2_ol
Run 4 for 111_trimethoxyethane
Run 5 for 1122_tetrachloroethane
Run 6 for 112_trichloro_122_trifluoroethane
Run 7 for 112_trichloroethane
Run 8 for 11_diacetoxyethane
Run 9 for 11_dichloroethane
Run 10 for 11_dichloroethene
Run 11 for 11_diethoxyethane
Run 12 for 11_difluoroethane
Run 13 for 1234_tetrachlorobenzene
Run 14 for 1235_tetrachlorobenzene
Run 15 for 123_trichlorobenzene
Run 16 for 123_trimethylbenzene
Run 17 for 1245_tetrachlorobenzene
Run 18 for 124_trichlorobenzene
Run 19 for 124_trimethylbenzene
Run 20 for 12_diacetoxyethane
Run 21 for 12_dibromoethane
Run 22 for 12_dichlorobenzene
Run 23 for 12_dichloroethane
Run 24 for 12_dichloropropane
Run 25 for 12_diethoxyethane
Run 26 for 12_dimethoxyethane
Run 27 for 12_ethanediol
Run 28 for 135_trichlorobenzene
Run 29 for 135_trimethylbenzene
Run 30 for 13_dichlorobenzene
Run 31 for 13_dic

Run 280 for di_n_butyl_ether
Run 281 for di_n_butylamine
Run 282 for di_n_propyl_ether
Run 283 for di_n_propyl_sulfide
Run 284 for di_n_propylamine
Run 285 for dibromomethane
Run 286 for dichloromethane
Run 287 for diethoxymethoxybenzene
Run 288 for diethyl_disulfide
Run 289 for diethyl_ether
Run 290 for diethyl_malonate
Run 291 for diethyl_succinate
Run 292 for diethyl_sulfide
Run 293 for diethylamine
Run 294 for diiodomethane
Run 295 for diisopropyl_ether
Run 296 for diisopropylamine
Run 297 for dimethoxymethane
Run 298 for dimethyl_disulfide
Run 299 for dimethyl_ether
Run 300 for dimethyl_sulfate
Run 301 for dimethyl_sulfide
Run 302 for dimethyl_sulfone
Run 303 for dimethyl_sulfoxide
Run 304 for dimethylamine
Run 305 for ethanamide
Run 306 for ethane
Run 307 for ethanethiol
Run 308 for ethanol
Run 309 for ethene
Run 310 for ethyl_acetate
Run 311 for ethyl_benzoate
Run 312 for ethyl_butanoate
Run 313 for ethyl_formate
Run 314 for ethyl_hexanoate
Run 315 for ethyl_pentanoate
Run 316 f